In [1]:
!pip install -q datasets transformers accelerate peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.3 MB/s eta 0:00:00


In [2]:
import pandas as pd

df = pd.read_csv("hf://datasets/BI55/MedText/medtext_2.csv")
print(df.columns)
print(df.iloc[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Index(['Prompt', 'Completion'], dtype='object')
Prompt        A 50-year-old male presents with a history of ...
Completion    This patient's history of recurrent kidney sto...
Name: 0, dtype: object


In [3]:
from datasets import Dataset

formatted_df = pd.DataFrame({
    "instruction": df["Prompt"],
    "input": "",
    "output": df["Completion"]
})

dataset = Dataset.from_pandas(formatted_df)
dataset = dataset.train_test_split(test_size=0.1)

dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1270
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 142
    })
})

In [4]:
from transformers import AutoTokenizer

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [6]:
def format_and_tokenize(example):
    full_prompt = f"Instruction: {example['instruction']}\nResponse:{example['output']}"
    return tokenizer(full_prompt, truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(format_and_tokenize, batched=False)

tokenized_dataset["train"][0]

Map:   0%|          | 0/1270 [00:00<?, ? examples/s]

Map:   0%|          | 0/142 [00:00<?, ? examples/s]

{'instruction': 'A 65-year-old female presents with a three-day history of sudden, episodic spinning sensation. Each episode lasts less than a minute and is associated with a change in head position. There is no hearing loss or tinnitus. What could be the potential diagnosis and the next step?',
 'input': '',
 'output': "The patient's symptoms are suggestive of benign paroxysmal positional vertigo (BPPV), which is characterized by brief episodes of mild to intense dizziness associated with specific changes in the position of the head. The next step would be to perform the Dix-Hallpike maneuver to confirm the diagnosis. If positive, the patient can be treated with vestibular rehabilitation exercises, such as the Epley maneuver.",
 'input_ids': [1,
  2799,
  4080,
  29901,
  319,
  29871,
  29953,
  29945,
  29899,
  6360,
  29899,
  1025,
  12944,
  22981,
  411,
  263,
  2211,
  29899,
  3250,
  4955,
  310,
  8327,
  29892,
  6010,
  397,
  293,
  10917,
  1076,
  4771,
  362,
  29889

In [7]:
from transformers import AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./tinyllama-medtext-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

/tmp/ipython-input-8-3420245632.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,1.592200
20,1.409600
30,1.317200
40,1.318300
50,1.244100
60,1.246400
70,1.264600
80,1.264700
90,1.223600
100,1.208500


TrainOutput(global_step=477, training_loss=1.2055442818055864, metrics={'train_runtime': 720.9481, 'train_samples_per_second': 5.285, 'train_steps_per_second': 0.662, 'total_flos': 1.212144773234688e+16, 'train_loss': 1.2055442818055864, 'epoch': 3.0})

In [9]:
model.save_pretrained("./tinyllama-medtext-lora")
tokenizer.save_pretrained("./tinyllama-medtext-lora")

('./tinyllama-medtext-lora/tokenizer_config.json',
 './tinyllama-medtext-lora/special_tokens_map.json',
 './tinyllama-medtext-lora/chat_template.jinja',
 './tinyllama-medtext-lora/tokenizer.model',
 './tinyllama-medtext-lora/added_tokens.json',
 './tinyllama-medtext-lora/tokenizer.json')

In [10]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="./tinyllama-medtext-lora",
    tokenizer="./tinyllama-medtext-lora",
    device=0
)

prompt = "Instruction: 65-year-old female presents with fatigue, weight loss, and elevated calcium levels.\nResponse:"

output = pipe(prompt, max_new_tokens=100, do_sample=True, temperature=0.7)[0]["generated_text"]

print(output)

Device set to use cuda:0


Instruction: 65-year-old female presents with fatigue, weight loss, and elevated calcium levels.
Response:A 65-year-old female with a history of hypertension, diabetes, and hyperlipidemia is at high risk for developing osteoporosis. This condition is characterized by thinning and weakening of bones, leading to an increased risk of fractures. She should be assessed for osteoporosis risk factors. Bone densitometry should be performed to determine the bone mineral density (BMD) of the


In [11]:
!pip install -q evaluate bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.6 MB/s eta 0:00:00


In [12]:
from transformers import pipeline

pipe = pipeline("text-generation", model="./tinyllama-medtext-lora", tokenizer="./tinyllama-medtext-lora", device=0)

eval_inputs = dataset["test"]

eval_results = []

for example in eval_inputs:
    prompt = f"Instruction: {example['instruction']}\nResponse:"
    output = pipe(prompt, max_new_tokens=100, do_sample=False)[0]["generated_text"]

    predicted_response = output.split("Response:")[-1].strip()

    eval_results.append({
        "instruction": example["instruction"],
        "prediction": predicted_response,
        "reference": example["output"]
    })

eval_results[0]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are no

{'instruction': 'A 25-year-old male with a known mutation for hereditary pancreatitis presents with abdominal pain, nausea, and vomiting. His amylase and lipase levels are elevated. What could be the diagnosis and the next steps?',
 'prediction': "The patient's symptoms and laboratory findings suggest pancreatitis, likely due to the mutation for hereditary pancreatitis. The next steps would be to confirm the diagnosis with a pancreatic enzyme assay, which would show elevated amylase and lipase levels. The patient should be started on pancreatic enzyme replacement therapy, which can help prevent further damage to the pancreas. If the patient is not respond",
 'reference': "Given this patient's genetic predisposition and his symptoms, he likely has a bout of acute pancreatitis. The next steps should include imaging to assess the pancreas and confirmation of the diagnosis. Management involves hydration, pain control, and monitoring for complications. As this patient is at increased risk f

In [13]:
!pip install rouge_score
!pip install bert_score

from evaluate import load

predictions = [ex["prediction"] for ex in eval_results]
references = [ex["reference"] for ex in eval_results]

rouge = load("rouge")
rouge_result = rouge.compute(predictions=predictions, references=references)

bertscore = load("bertscore")
bertscore_result = bertscore.compute(predictions=predictions, references=references, lang="en")

print("ROUGE Score:", rouge_result)
print("BERT Score:", bertscore_result)

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=bca9963d13ab589c5567168d6e4b1e75cbad0d95b3eebbf2f0e0f36456780197
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE Score: {'rouge1': np.float64(0.35355941128014173), 'rouge2': np.float64(0.10806436220849416), 'rougeL': np.float64(0.233045969406719), 'rougeLsum': np.float64(0.23431460009797003)}
BERT Score: {'precision': [0.8817970752716064, 0.889982283115387, 0.8707969784736633, 0.8952793478965759, 0.9333521723747253, 0.8645368814468384, 0.8583776354789734, 0.9012960195541382, 0.9136055707931519, 0.8901335000991821, 0.8600272536277771, 0.9077094197273254, 0.8670984506607056, 0.8961936235427856, 0.8725035190582275, 0.8579686880111694, 0.8679529428482056, 0.8720717430114746, 0.8829177618026733, 0.8738918900489807, 0.8798283338546753, 0.8702229857444763, 0.8811171054840088, 0.8846186399459839, 0.88496994972229, 0.8567003011703491, 0.8961440920829773, 0.8779215812683105, 0.8983646631240845, 0.8789613246917725, 0.8826195597648621, 0.8910906314849854, 0.8728181719779968, 0.8855583667755127, 0.8640170097351074, 0.8463760614395142, 0.8980406522750854, 0.8624855279922485, 0.8755931854248047, 0.8902221